In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# 1. Instalação de pacotes
!pip install -U "unsloth[torch]" -q
!pip install "transformers==4.51.3" "datasets" "peft" "trl" "bitsandbytes" accelerate -q

In [4]:
# 2. Importações principais
from unsloth import FastLanguageModel
from datasets import load_dataset
from transformers import TrainingArguments, DataCollatorForLanguageModeling
import torch

# 3. Carrega modelo 4-bit + tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = 2048,
    dtype = torch.float16,
    load_in_4bit = True,
)

# 4. Aplica LoRA para permitir treino
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
)

# 5. Carrega o dataset no formato jsonl
# O arquivo deve ter linhas como:
# {"text": "Gere uma descrição para o produto: Tênis XYZ\nDescrição: Tênis de corrida com..."}
dataset = load_dataset("json", data_files="/content/drive/MyDrive/Colab Notebooks/Fine_Tuning_Amazon_Titles_v3/dados/trn_Amazon_titles.jsonl")["train"]


==((====))==  Unsloth 2025.5.7: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.5.7 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [5]:
# 6. Tokeniza o dataset manualmente
def tokenize(example):
    return tokenizer(
        example["text"],
        truncation=True,
        padding=False,
        max_length=2048,
    )

tokenized_dataset = dataset.map(tokenize, batched=True)

Map:   0%|          | 0/210551 [00:00<?, ? examples/s]

In [6]:
# 7. Data collator para linguagem causal (sem masked language model)
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

In [7]:
# 8. Argumentos do treinamento
training_args = TrainingArguments(
    output_dir="modelo_lora_final",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=20,
    max_steps=300,  # Ajuste conforme o tamanho do seu dataset
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,
    save_steps=100,
    save_total_limit=2,
    report_to="none"
)

In [10]:
from transformers import Trainer

# 9. Treina o modelo
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

<ipython-input-10-84d49e793c06>:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 210,551 | Num Epochs = 1 | Total steps = 300
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,2.634300
20,2.166000
30,2.215800
40,2.142200
50,2.147500
60,2.256900
70,2.184400
80,2.130400
90,2.153600
100,2.105000


TrainOutput(global_step=300, training_loss=2.1778425470987957, metrics={'train_runtime': 1015.1519, 'train_samples_per_second': 2.364, 'train_steps_per_second': 0.296, 'total_flos': 2.095505551373107e+16, 'train_loss': 2.1778425470987957, 'epoch': 0.011398609369656901})

In [12]:
# 10. Salva o modelo treinado
model.save_pretrained("modelo_lora_final")
tokenizer.save_pretrained("modelo_lora_final")

('modelo_lora_final/tokenizer_config.json',
 'modelo_lora_final/special_tokens_map.json',
 'modelo_lora_final/tokenizer.json')

In [13]:
output_directory = "/content/drive/MyDrive/Colab Notebooks/Fine_Tuning_Amazon_Titles_v3/modelo_lora_final" # Substitua pelo seu caminho real
model.save_pretrained(output_directory)
tokenizer.save_pretrained(output_directory)

('/content/drive/MyDrive/Colab Notebooks/Fine_Tuning_Amazon_Titles_v3/modelo_lora_final/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/Fine_Tuning_Amazon_Titles_v3/modelo_lora_final/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/Fine_Tuning_Amazon_Titles_v3/modelo_lora_final/tokenizer.json')